# DATA PREPARATION

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data/training_set_VU_DM.csv')
print("the shape of the data ",df.shape)
df.head()

the shape of the data  (4958347, 54)


,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,NaN,NaN,NaN,NaN,-1.0,0.0,5.0,0,NaN,0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0


In [4]:
def remove_outliers_zscore(df, columns, threshold=0.5):
    # Calculate the z-scores for the specified columns
    z_scores = df[columns].apply(lambda x: (x - x.mean()) / x.std())
    
    # Identify the rows containing outliers based on the threshold
    outlier_rows = z_scores.abs().apply(lambda x: any(abs(val) > threshold for val in x), axis=1)
    
    # Get the outliers and the cleaned DataFrame
    outliers = df[outlier_rows]
    df_cleaned = df[~outlier_rows]
    
    return df_cleaned, outliers

columns = ['price_usd','comp3_rate_percent_diff',
            'comp8_rate_percent_diff','comp5_rate_percent_diff','comp4_rate_percent_diff']

cleaned_df, outliers = remove_outliers_zscore(df, columns, threshold=0.5)

In [5]:
# to fill the missing value with first quartile
def fill_missing_values_with_first_quartile(df, columns_to_fill):
    # Loop through each column and fill missing values with the first quartile of the respective country
    for column in columns_to_fill:
        # Group the data by country and calculate the first quartile of the column
        first_quartile_by_country = round(df.groupby("prop_country_id")[column].quantile(0.25))
        # Fill missing values in the column with the first quartile of the respective country
        df[column].fillna(df["prop_country_id"].map(first_quartile_by_country), inplace=True)
    
    return df

# Define a list of columns to fill missing values for
columns_to_fill = ['visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_review_score', 'prop_location_score2',
                   'srch_query_affinity_score', 'orig_destination_distance', 'comp1_rate', 'comp1_inv', 
                   'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv', 'comp2_rate_percent_diff', 'comp3_rate', 
                   'comp3_inv', 'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv', 'comp4_rate_percent_diff', 
                   'comp5_rate', 'comp5_inv', 'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
                   'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv', 'comp7_rate_percent_diff',
                   'comp8_rate', 'comp8_inv', 'comp8_rate_percent_diff']

new_df = fill_missing_values_with_first_quartile(cleaned_df,columns_to_fill)

/var/folders/cl/l2pgxhp51rs57jb2mpt6g5vc0000gn/T/ipykernel_20645/3780794415.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column].fillna(df["prop_country_id"].map(first_quartile_by_country), inplace=True)


In [6]:
new_df.isnull().sum()

srch_id                              0
date_time                            0
site_id                              0
visitor_location_country_id          0
visitor_hist_starrating           2710
visitor_hist_adr_usd              2710
prop_country_id                      0
prop_id                              0
prop_starrating                      0
prop_review_score                    0
prop_brand_bool                      0
prop_location_score1                 0
prop_location_score2                 2
prop_log_historical_price            0
position                             0
price_usd                            0
promotion_flag                       0
srch_destination_id                  0
srch_length_of_stay                  0
srch_booking_window                  0
srch_adults_count                    0
srch_children_count                  0
srch_room_count                      0
srch_saturday_night_bool             0
srch_query_affinity_score        20516
orig_destination_distance

In [7]:
# fill the missing values with forward filling method 
new_df = new_df.fillna(method='ffill')
# check if still some missing values
new_df.isnull().sum()

srch_id                         0
date_time                       0
site_id                         0
visitor_location_country_id     0
visitor_hist_starrating         0
visitor_hist_adr_usd            0
prop_country_id                 0
prop_id                         0
prop_starrating                 0
prop_review_score               0
prop_brand_bool                 0
prop_location_score1            0
prop_location_score2            0
prop_log_historical_price       0
position                        0
price_usd                       0
promotion_flag                  0
srch_destination_id             0
srch_length_of_stay             0
srch_booking_window             0
srch_adults_count               0
srch_children_count             0
srch_room_count                 0
srch_saturday_night_bool        0
srch_query_affinity_score       0
orig_destination_distance       0
random_bool                     0
comp1_rate                      0
comp1_inv                       0
comp1_rate_per

In [8]:
# drop the features still have a lot of missing values
new_df=new_df.drop(['gross_bookings_usd'],axis=1)
# new_df=new_df.drop(['comp6_rate','comp6_inv','comp6_rate_percent_diff','gross_bookings_usd'],axis=1)

In [9]:
# Features engineer
data = new_df
# convert date_time to year, month, day, and hour
data['date_time'] = pd.to_datetime(data['date_time'])
data['year'] = data['date_time'].dt.year
data['month'] = data['date_time'].dt.month
data['day'] = data['date_time'].dt.day
data['hour'] = data['date_time'].dt.hour
data = data.drop('date_time', axis=1)

# create new features
data['log_price_diff'] = data['prop_log_historical_price'] - np.log(data['price_usd']+1)

data['star_rank'] = data.groupby('visitor_location_country_id')['prop_starrating'].rank(method='dense', ascending=False)

# Calculate average price by country and star rating
avg_price = data.groupby(['prop_country_id', 'prop_starrating'])['price_usd'].transform('mean')
# Calculate price difference and Add price difference as a new feature
data['price_diff'] = data['price_usd'] - avg_price


# To combine the prop_location_score1 and prop_location_score2 columns to create a new feature that captures 
# the overall location score of the hotel, we can simply add these two columns
data['location_score'] = data['prop_location_score1'] + data['prop_location_score2']

print(data.shape)
data.head()

(4955367, 60)


,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,click_bool,booking_bool,year,month,day,hour,log_price_diff,star_rank,price_diff,location_score
0,1,12,187,3.0,101.0,219,893,3,3.5,1,...,0,0,2013,4,4,8,0.288733,3.0,-26.137525,2.8738
1,1,12,187,3.0,101.0,219,10404,4,4.0,1,...,0,0,2013,4,4,8,-0.115982,2.0,-39.189522,2.2149
2,1,12,187,3.0,101.0,219,21315,3,4.5,1,...,0,0,2013,4,4,8,-0.277391,3.0,48.892475,2.2245
3,1,12,187,3.0,101.0,219,27348,2,4.0,1,...,0,0,2013,4,4,8,-2.013193,4.0,517.518263,2.8425
4,1,12,187,3.0,101.0,219,29604,4,3.5,1,...,0,0,2013,4,4,8,-0.043833,2.0,-66.349522,2.7641


# MODELING AND EVALUATION

## Collaborative Filtering

In [10]:
test_set = pd.read_csv('test_set_VU_DM.csv', nrows=100 )#, skiprows=range(1, 4400000))

In [11]:
# get a subset of the training data similar to the query 
def subset_of_data(data,
                   prop_country_id,
                   srch_destination_id,
                   srch_length_of_stay=None,
                   srch_room_count=None,
                   srch_adults_count=None,
                   srch_children_count= None):    
    
    new_df = data[data['prop_country_id'] == prop_country_id]
    new_df1 = new_df[new_df['srch_destination_id'] == srch_destination_id]
    new_df2 = new_df1[new_df1['srch_length_of_stay'] == srch_length_of_stay]
    new_df3 = new_df2[new_df2['srch_room_count'] == srch_room_count]
    new_df4 = new_df3[new_df3['srch_adults_count'] == srch_adults_count]
    new_df5= new_df4[new_df4['srch_children_count'] == srch_children_count]
    return new_df5

In [12]:
def create_train_matrix(new_df,instance):    
    # Concatenate the dataframes vertically (along the rows)
    new_df = pd.concat([new_df, instance], axis=0)
    # Get unique prop_id values
    unique_props = new_df['prop_id'].unique()

    # Create a new DataFrame with prop_id as columns and srch_id as index
    matrix = pd.DataFrame(0, index=new_df['srch_id'].unique(), columns=unique_props)

    # Fill the matrix with the interaction or rating values
    for _, row in new_df.iterrows():
        srch_id = row['srch_id']
        prop_id = row['prop_id']
    # rating1 = row['click_bool']  # You can define your own rating metric
        rating = row['click_bool'] + 10 * row['booking_bool'] - (row['price_usd'] / new_df['price_usd'].max()) +\
        + row['prop_review_score']-3 - row['promotion_flag']
        
        matrix.loc[srch_id, prop_id] = rating
    return matrix

In [13]:
def sum_sort(matrix):
    matrix1 = matrix.sum(axis=0)
    sorted_values = matrix1.sort_values(ascending=False)
    return sorted_values

In [14]:
def get_instance(data,srch_id):
    instance = data[data['srch_id'] == srch_id].reset_index(drop=True)
    return instance
# instance = get_instance(test_set,srch_id=10)

In [15]:
with open("result-1.csv", "w+") as file:
    file.write("srch_id,prop_id")
    
    for inst in test_set['srch_id'].unique():
        instance = get_instance(test_set,srch_id=inst)
        reco_hotel = None
        for i, row in instance.iterrows():
            test = subset_of_data(data= data,
                                  prop_country_id= row['prop_country_id'],
                                  srch_destination_id= row["srch_destination_id"],
                                  srch_length_of_stay= row['srch_length_of_stay'],
                                   srch_room_count= row['srch_room_count'],
                                 srch_adults_count= row['srch_adults_count'],
                                 srch_children_count= row['srch_children_count']
                                 )
            matrix = create_train_matrix(test,instance)
            reco_hotel = sum_sort(matrix)
            reco_hotel = (reco_hotel.index)
            break

        for i in range(len(instance)):
            
            file.write(f"\n{inst},{reco_hotel[i]}")

In [16]:
data.columns

Index(['srch_id', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv'

## Lambda Rank

In [ ]:
features = ['prop_location_score2', 'price_usd', 'promotion_flag']

In [ ]:
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score

# df = pd.read_csv("/kaggle/input/vu-dmt-assigment-2-2023/training_set_VU_DM.csv")
target = 'booking_bool'
train_df, val_df = train_test_split(data, test_size=0.2)
lgb_train = lgb.Dataset(train_df[features], label=train_df[target], group=train_df.groupby('srch_id').size())

params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'ndcg_eval_at': [5],
    'learning_rate': 0.1,
    'num_leaves': 31,
    'verbose': 1
}

gbm = lgb.train(params, lgb_train, num_boost_round=100)
val_pred = gbm.predict(val_df[features])
val_labels = val_df[target]
ndcg = ndcg_score([val_pred], [val_labels], k=5)
print("NDCG@5:", ndcg)


def create_submission_file():
    test_df = pd.read_csv("data/test_set_VU_DM.csv")
    test_pred = gbm.predict(test_df[features])
    properties_to_rank = test_df[['srch_id', 'prop_id', 'prop_location_score2', 'price_usd', 'promotion_flag']].copy()
    properties_to_rank["score"] = test_pred
    aggregated_scores = properties_to_rank.groupby(['srch_id', 'prop_id'])['score'].mean().reset_index()
    ranked_properties = aggregated_scores.sort_values(['srch_id', 'score'], ascending=[True, False])

    file = open("result-2.csv", "w+")
    file.write("srch_id,prop_id")
    for srch_id, group in ranked_properties.groupby('srch_id'):
        for i, row in group.iterrows():
            file.write(f"\n{srch_id},{int(row['prop_id'])}")